## הגדרות ומקדים

### ספריות

In [1]:
import os
import sys
import fiona
import pandas as pd
import geopandas as gpd
from openpyxl import load_workbook

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### העלת משתנים להרצת הקוד

In [6]:
path = os.getcwd()

software_root_folder = os.path.dirname(path)

software_folder_location = r'{}\current'.format(software_root_folder)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_folder_location))

forecast_version_folder_location=df_inputs_outputs['location'][1]

output_location=df_inputs_outputs['location'][2]

sys.path.append(software_root_folder)

'C:\\Users\\dpere\\Documents\\JTMT\\forecast_by_version\\V4\\BASE_YEAR'

## פונקציות

In [4]:
def drop_geo(geoDF):
    geoDF = geoDF.drop(columns='geometry')
    return geoDF

In [5]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [6]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [7]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [8]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [9]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [10]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df

## להריץ תלמידים

In [11]:
%run "students_current_year_230706.ipynb"

C:\Users\dpere\AppData\Local\Temp\ipykernel_3432\3525234411.py:14: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  student_yeshiva=student_yeshiva[col].pivot_table(index='Taz_num',aggfunc=sum)[[
C:\Users\dpere\AppData\Local\Temp\ipykernel_3432\2976863300.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  taz['student_gov']=drop_geo(student_gov_taz).pivot_table(index='Taz_num',aggfunc=sum)[['num_of_students']]
C:\Users\dpere\AppData\Local\Temp\ipykernel_3432\3559353202.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provi

## להעלות שכבות

In [12]:
path=r'{}\background_files\EMP_KIBOLET.gdb'.format(software_folder_location)
EMP_kibolet=up_load_gdb(path,'EMP_kibolet')

In [13]:
EMP_kibolet=EMP_kibolet.fillna(0)

In [14]:
EMP_kibolet['emp_without_palestin']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

In [15]:
emp_without_palestin_not_okev=EMP_kibolet['emp_without_palestin'].sum().item()

In [16]:
emp_Education=taz.query('main_secto!="Palestinian"')['emp_Education'].sum().item()

## הוספת סטודנטים לאוכלוסיה ולהתפלגות גילים

In [17]:
student_dorms=up_load_shp(r'{}\background_files\student_dorms.shp'.format(software_folder_location))


In [18]:
# taz['student_dorms']=gpd.sjoin(taz[['geometry']].reset_index(),student_dorms).pivot_table(index='Taz_num',aggfunc=sum)[['student_nu']]
taz['student_dorms']=gpd.sjoin(taz[['geometry']].reset_index(),student_dorms)[['Taz_num','student_nu']].pivot_table(index='Taz_num',aggfunc=sum)

C:\Users\dpere\AppData\Local\Temp\ipykernel_3432\1459382123.py:2: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  taz['student_dorms']=gpd.sjoin(taz[['geometry']].reset_index(),student_dorms)[['Taz_num','student_nu']].pivot_table(index='Taz_num',aggfunc=sum)


In [19]:
taz=taz.fillna(0)

In [20]:
taz['pop_without_dorms_yeshiva']=taz['pop']

In [21]:
taz['pop']=taz['pop']+taz['student_dorms']

In [22]:
taz['pop_20_just_from_aprt']=taz['pop_20']

In [23]:
taz['pop_25_just_from_aprt']=taz['pop_25']

In [24]:
taz['pop_20']=taz['pop_20']+taz['student_dorms']*0.6

In [25]:
taz['pop_25']=taz['pop_25']+taz['student_dorms']*0.4

In [26]:
col=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## הוספת תלמידי ישיבה (באזורים חרדיים) אל כמות אוכלוסיה,התפלגות גילים 

In [27]:
taz['pop_15_just_from_aprt']=taz['pop_15']

In [28]:

taz.loc[taz['main_secto']=="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

In [29]:
col=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_15_just_from_aprt
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## כימות מועסקים במרחב צתאל

In [30]:
pre_woman=0.5

In [31]:
pre_man=1-pre_woman

In [32]:
work_age=[ 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60']

In [33]:
under_work_age=[ 'pop_15', 'pop_20']

In [34]:
over_work_age=[ 'pop_65', 'pop_70', 'pop_75up']

In [35]:
taz['work_age']=taz[work_age].sum(axis=1)

taz['under_work_age']=taz[under_work_age].sum(axis=1)

taz['over_work_age']=taz[over_work_age].sum(axis=1)

In [36]:
sector='U_Orthodox'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.75+taz['work_age']*pre_man*0.55
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.07+taz['under_work_age']*pre_man*0.09
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09

sector='Jewish'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.9+taz['work_age']*pre_man*0.92
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.20+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.2+taz['over_work_age']*pre_man*0.15

sector='Arab'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.25+taz['work_age']*pre_man*0.7
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.2+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09

sector='arabs_behined_seperation_wall'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.25+taz['work_age']*pre_man*0.7
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.2+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09


## חישוב אבטלה

In [37]:
taz['pop_emp_employed']=0

In [38]:
taz.loc[taz['main_secto']=="U_Orthodox",'pop_emp_employed']=taz['pop_emp']*0.97

C:\Users\dpere\AppData\Local\Temp\ipykernel_3432\2869267454.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1058.5125      925.0696     1368.9416     1401.4269      900.4801
  662.6458      925.0599      932.1215      931.33828983  997.1115
  567.256       832.8517     1120.5246      658.6882      798.3682
 1469.065      1668.1284      882.8552     1070.52809036  841.5235
  547.8269      625.63533771  421.00136291 1126.267       588.8191
  497.06827099  192.98137702  550.82128767  652.67722429    0.
  199.05712       0.         1029.46150669  958.554      1089.47169269
  258.27169331  316.24394151  868.35219351  381.68004435  346.99131233
 1150.0223      711.9509      926.2918      586.98056974  970.0056804
  242.39101412  485.02180886  448.62910236  700.57393582  187.79718023
  912.98938137  491.97901778    0.           31.82368444  345.50725071
  424.43994929  589.5392783   674.3028217  1079.7244229   492.62

In [39]:
taz.loc[taz['main_secto']=="Jewish",'pop_emp_employed']=taz['pop_emp']*0.98

In [40]:
taz.loc[taz['jew']==0,'pop_emp_employed']=taz['pop_emp']*0.95

## חישוב יוממות

In [41]:
commuting=up_load_shp(r'{}\background_files\commuting_230712.shp'.format(software_folder_location))


In [42]:
taz['commuting']=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],commuting[['commuting', 'geometry']]).set_index('Taz_num')['commuting']/100

In [43]:
taz['pop_emp_employed_out_of_jtmt_area']=taz['pop_emp_employed']*taz['commuting']

In [44]:
emp_left_jtmt_area=taz['pop_emp_employed_out_of_jtmt_area'].sum().item()

In [45]:
round(emp_left_jtmt_area,-3)

94000.0

In [46]:
emp_from_jtmt_area=taz['pop_emp_employed'].sum().item()-emp_left_jtmt_area

In [47]:
emp_in_jtmt_area=emp_from_jtmt_area*1.07 #יוממות פנימה מחוץ למרחב

In [48]:
round(emp_in_jtmt_area,-3)

510000.0

In [49]:
emp_in_jtmt_area_without_mobile=emp_in_jtmt_area*0.95 #הפחחת עובדים ניידים

In [50]:
round(emp_in_jtmt_area_without_mobile,-3)

484000.0

In [51]:
emp_okev=emp_in_jtmt_area_without_mobile-emp_Education-emp_without_palestin_not_okev

## פיזור עוקב

In [52]:
EMP_kibolet['geometry_buff']=EMP_kibolet.buffer(250)

In [53]:
EMP_buffer=EMP_kibolet.set_geometry('geometry_buff').query('kayim_emp>0')

In [54]:
EMP_buffer=EMP_buffer.dissolve()[['geometry_buff']]

In [55]:
taz['taz_area']=taz.area

In [56]:
emp_buffer_taz=gpd.overlay(EMP_buffer,taz.reset_index())

In [57]:
emp_buffer_taz['emp_samll_area']=emp_buffer_taz.area

In [58]:
emp_buffer_taz['emp_pre_from_taz']=emp_buffer_taz['emp_samll_area']/emp_buffer_taz['taz_area']

In [59]:
taz_num_no_need_okev=emp_buffer_taz.loc[emp_buffer_taz['emp_pre_from_taz']>0.4].Taz_num.to_list()

In [60]:
taz['okev']=0
taz.loc[(~taz.index.isin(taz_num_no_need_okev))&(taz['pop']>0)&(taz['main_secto']!="Palestinia"),'okev']=1

In [61]:
taz.loc[taz['main_secto']=="arabs_behined_seperation_wall",'okev']=1

In [62]:
city_muni=['מודיעין עילית','בית שמש','ירושלים','מודיעין - מכבים - רעות']

## מקדם עוקב

In [63]:
okev_level=0.15

In [64]:
arab_city=taz.query('jew==0 & okev==1')['aprt_20'].sum().item()

In [65]:
arab_sub=0

In [66]:
ou_sub=taz.query('main_secto=="U_Orthodox" & ~Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [67]:
ou_city=taz.query('main_secto=="U_Orthodox" & Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [68]:
general_city=taz.query('main_secto=="Jewish" & Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [69]:
general_sub=taz.query('main_secto=="Jewish" & ~Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [70]:
base_okev_factor=(emp_okev-(okev_level*general_city-okev_level*(ou_sub+arab_city)-2*okev_level*arab_sub))/(general_sub+general_city+ou_city+ou_sub+arab_city+arab_sub)

In [71]:
base_okev_factor=round(base_okev_factor,3)

In [72]:
data = [['Jewish', base_okev_factor,base_okev_factor+okev_level],['arab', base_okev_factor-okev_level,base_okev_factor], ['U_Orthodox',base_okev_factor-2*okev_level,base_okev_factor-okev_level]]
 
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['sector', 'sub','city'])
okev_factor=df.set_index('sector')
# okev_factor.to_excel(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\output\okev_factor.xlsx')
okev_factor.to_excel(r'{}\Intermediates\okev_factor.xlsx'.format(software_folder_location))

In [73]:
#okev_factor  ##אם ארצה לראות את הטבלה

In [74]:
taz['emp_okev']=0

In [75]:
taz.loc[(taz['jew']==0)&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['arab','city']

C:\Users\dpere\AppData\Local\Temp\ipykernel_3432\2496725524.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2536.68600793  663.95841106 1454.6965302  1162.87507074  461.71011345
  181.0381023   563.90819034  387.9984432  1427.00791646  592.29263186
  116.29186861  302.98046383  356.89141041  152.32446631   97.17332518
  564.66626503  405.94487016  577.96949449  172.90532509  439.25456267
  636.55659017  568.59927227  589.87006481  458.39639966  500.73884984
 1018.37538885  563.732      2552.23499218  816.3985081   586.78536878
  316.21990544  316.56324671  244.08331939  722.5343934   781.28637177
  383.56703406  700.3088952   678.22662547  503.27967819  580.20120115
  865.88369491 2152.10293558   49.66809796  427.22990387  427.66448377
  266.99793209  284.79444143  120.61427701  585.40124266  206.79138999
  198.92284348  988.44736647  526.70374934]' has dtype incompatible with int64, please explicitly cast to 

In [76]:
taz.loc[(taz['main_secto']=="U_Orthodox")&(~taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['U_Orthodox','sub']

In [77]:
taz.loc[(taz['main_secto']=="U_Orthodox")&(taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['U_Orthodox','city']

In [78]:
taz.loc[(taz['main_secto']=="Jewish")&(taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['Jewish','city']

In [79]:
taz.loc[(taz['main_secto']=="Jewish")&(~taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['Jewish','sub']

In [80]:
taz.emp_okev.sum().item()-emp_okev

-1816.5115199011634

## פיצול שכבת לא עוקב לאזורי תנועה

In [81]:
EMP_kibolet['emp_area']=EMP_kibolet.area

In [82]:
EMP_kibolet['ID']=EMP_kibolet.index

In [83]:
EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)

C:\Users\dpere\AppData\Local\Temp\ipykernel_3432\35171637.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 270 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)


In [84]:
EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

In [85]:
EMP_kibolet_by_taz['pre_small_area_emp']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['emp_area']

In [86]:
EMP_kibolet_by_taz['pre_small_area_taz']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['taz_area']

In [87]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.query('(pre_small_area_taz >0.7) | (pre_small_area_emp >0.1)').drop(columns='geometry_buff')

In [88]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.set_index('ID')

In [89]:
# EMP_kibolet_by_taz['id_area_for_pre']=EMP_kibolet_by_taz.reset_index().groupby(by='ID').sum()['small_area']
EMP_kibolet_by_taz['id_area_for_pre']=EMP_kibolet_by_taz.reset_index().groupby(by='ID')['small_area'].sum()

In [90]:
EMP_kibolet_by_taz['prec_from_id']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['id_area_for_pre']

In [91]:
EMP_kibolet_by_taz['emp_without_palestin_in_taz']=EMP_kibolet_by_taz['emp_without_palestin']*EMP_kibolet_by_taz['prec_from_id']

In [92]:
taz['emp_not_okev']=drop_geo(EMP_kibolet_by_taz).pivot_table(index='Taz_num',aggfunc=sum)[['emp_without_palestin_in_taz']]

C:\Users\dpere\AppData\Local\Temp\ipykernel_3432\1761427109.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  taz['emp_not_okev']=drop_geo(EMP_kibolet_by_taz).pivot_table(index='Taz_num',aggfunc=sum)[['emp_without_palestin_in_taz']]


In [93]:
taz=taz.fillna(0)

In [94]:
taz['total_emp']=taz['emp_not_okev']+taz['emp_okev']+taz['emp_Education']

In [95]:
taz.query('main_secto!="Palestinian"')['total_emp'].sum()-emp_in_jtmt_area_without_mobile

-1816.5115199011634

## תוצאות

In [96]:
# taz.groupby(by='zonetype').sum()[['total_emp']]#.loc[['Jerusalem','Judea and Samaria']]
taz.groupby(by='zonetype')[['total_emp']].sum()#.loc[['Jerusalem','Judea and Samaria']]

,total_emp
zonetype,
Jerusalem,372287.064
Judea and Samaria,82187.101
Ramla,28207.725


In [97]:
# taz.groupby(by='Muni_Heb').sum()[['total_emp']].loc[['ירושלים','בית שמש','ביתר עילית','מודיעין - מכבים - רעות','מעלה אדומים','מודיעין עילית']]
taz.groupby(by='Muni_Heb')[['total_emp']].sum().loc[['ירושלים','בית שמש','ביתר עילית','מודיעין - מכבים - רעות','מעלה אדומים','מודיעין עילית']]

,total_emp
Muni_Heb,
ירושלים,324011.114
בית שמש,26993.914
ביתר עילית,7183.027
מודיעין - מכבים - רעות,25599.930
מעלה אדומים,7633.667
מודיעין עילית,10778.885


In [98]:
# taz.groupby(by=['Muni_Heb','main_secto']).sum()[['total_emp']].loc[['ירושלים']]
taz.groupby(by=['Muni_Heb','main_secto'])[['total_emp']].sum().loc[['ירושלים']]

total_emp
Muni_Heb main_secto                              
ירושלים  Arab                           57190.378
         Jewish                        197846.853
         U_Orthodox                     55886.933
         arabs_behined_seperation_wall  13086.951

## התפלגות ענפי תעסוקה

In [99]:
emp_category_type=up_load_shp(r'{}\background_files\emp_category_type.shp'.format(software_folder_location)).fillna(0)

In [100]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

In [101]:
emp_category_type[col]=emp_category_type[col]/100

In [102]:
emp_category_type=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],emp_category_type).set_index('Taz_num')

In [103]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

In [104]:
taz.join(emp_category_type[col]).query('(emp_not_okev>0 | emp_okev>0 )& agri.isna() & main_secto!="Palestinian" ')#[['geometry']].to_file(r'\\FILE-SRV\Jtmt\projections_team\תכניות אסטרטגיות לתחבורה ירושלים\תחזיות 2050\קבלת מידע\התפלגות ענפי כלכלה תעסוקה\emp_category_type_to_delet.shp')

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_15_just_from_aprt,work_age,under_work_age,over_work_age,pop_emp,pop_emp_employed,commuting,pop_emp_employed_out_of_jtmt_area,taz_area,okev,emp_okev,emp_not_okev,total_emp,agri,Indus,Com_hotel,Business,Public
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [105]:
taz=taz.join(emp_category_type[col]).fillna(0)

In [106]:
for i in col:
    taz['{}'.format(i)]=taz['{}'.format(i)]*(taz['emp_not_okev']+taz['emp_okev'])

In [107]:
taz['check_emp_sum']=taz[col].sum(axis=1)-(taz['emp_not_okev']+taz['emp_okev'])

In [108]:
taz.query('check_emp_sum>10 |check_emp_sum<-10')

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_15_just_from_aprt,work_age,under_work_age,over_work_age,pop_emp,pop_emp_employed,commuting,pop_emp_employed_out_of_jtmt_area,taz_area,okev,emp_okev,emp_not_okev,total_emp,agri,Indus,Com_hotel,Business,Public,check_emp_sum
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## הוספת סטודנטים אל כמות משקי בית

In [109]:
taz=taz.fillna(0)

In [110]:
taz['hh']=taz['aprt_20']+taz['student_dorms']/1.5

## הוספת תלמידי ישיבה (באזורים לא חרדיים) אל כמות אוכלוסיה,התפלגות גילים ומשקי בית (לכלל תלמידי הישיבה)

In [111]:
taz['hh']=taz['hh']+taz['yeshiva_dorms_pop_sum']

In [112]:
taz.loc[taz['main_secto']!="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

## הוספת אוכלוסיה פלסטינאית

In [113]:
path=r'{}\background_files'.format(software_folder_location)
taz_demo_pls_2020=up_load_df(path,'taz_demo_pls_2020_and_pre_growth_till_2050')[['Taz_num','pop_2020']].set_index('Taz_num')

In [114]:
col=['pop',
'pop_0',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_5',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'emp_Education',
'pop_emp_employed',
'total_emp',
'agri',
'Indus',
'Com_hotel',
'Business',
'Public',
'hh']

In [115]:
taz=pd.concat([taz.reset_index(),taz_border.loc[taz_border['main_secto']=='Palestinian']]).set_index('Taz_num').fillna(0)

In [116]:
taz.loc[taz['main_secto']=='Palestinian','pop']=taz['pop']+taz_demo_pls_2020['pop_2020']

## ייצוא לבקרה

In [117]:
taz=taz.reset_index()

In [118]:
pre_yeshiva_from_20_30_age=taz[['student_yeshiva_with_add_from_old']].sum().sum()/taz.query('jew==1')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

In [119]:
pre_yeshiva_from_20_30_age

0.524143764307828

In [120]:
taz.main_secto.unique()

array(['U_Orthodox', 'arabs_behined_seperation_wall', 'Arab', 'Jewish',
       'Palestinian'], dtype=object)

In [121]:
pre_uni_from_20_30_age=taz[['uni_students']].sum().sum()/taz.query('main_secto!="Palestinian"')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

In [122]:
pre_uni_from_20_30_age

0.3034760359349936

In [123]:
col=['Taz_num','Agg_taz_nu','Taz_name','Muni_Heb','jeru_metro',
 'zonetype',
 'in_jerusal','main_secto','aprt_20','student_dorms','student_yeshiva_with_add_from_old','hh','pop_without_dorms_yeshiva',
 'pop', 'hh_size','pop_emp_employed',
 'commuting','student_toddlers',
 'student_gov','cbs_muni_student_left_by_pre_of_demand_left','cbs_muni_student_left_by_pre_of_demand','student_chardi_not_gov',
 'student_arab_not_gov','student','uni_students',
 'student_yeshiva','yeshiva_dorms_pop_15',
 'yeshiva_dorms_pop_20',
 'yeshiva_dorms_pop_25',
 'yeshiva_dorms_pop_sum',
 'add_from_old_student_yeshiva',
 'add_from_kollim_demand', 'emp_from_uni_student','emp_from_Yeshiva_student','emp_from_student',
 'emp_Education','okev',
 'emp_okev',
 'emp_not_okev','total_emp',
 'agri',
 'Indus',
 'Com_hotel',
 'Business',
 'Public']

In [124]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [125]:
taz[col].to_excel(r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(output_location,file_date),index=False)

## ייצוא בפורמט של מודלים

In [126]:
taz['yosh']=0

In [127]:
taz.loc[taz['zonetype']=='Judea and Samaria','yosh']=1

In [128]:
taz['jerusalem_city']=0

In [129]:
taz.loc[taz['in_jerusal']=='yes','jerusalem_city']=1

In [130]:
taz=taz.reset_index()

In [131]:
col=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_city',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'pop_emp_employed']

In [132]:
col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [133]:
taz=taz[col]

In [134]:
taz.columns=col_new_name

In [135]:
taz.to_excel(r'{}\forecast_2020_{}.xlsx'.format(output_location,file_date),index=False)

In [137]:
#load excel file
workbook = load_workbook(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))

#open workbook
sheet = workbook.active

#modify the desired cell
sheet["B4"] = forecast_version_folder_location
sheet["B5"] = False
sheet["B6"] = ''

#save the file
workbook.save(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))

## בקרות לוגיות

In [672]:
taz.loc[taz['TAZ']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [673]:
taz.yosh.unique()

array([0, 1], dtype=int64)

In [674]:
taz.in_jerusalem_metropolin.unique()

array([1., 0.])

In [675]:
taz.jerusalem_city.unique()

array([1, 0], dtype=int64)

In [676]:
taz.sector.unique()

array(['U_Orthodox', 'arabs_behined_seperation_wall', 'Arab', 'Jewish',
       'Palestinian'], dtype=object)

In [677]:
col=['hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [678]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]<0]['TAZ'])

In [679]:
taz_to_check

[]

In [680]:
taz.loc[~(taz['hh_total']<=taz['pop'])]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [681]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['hh_total']>0].loc[taz['pop']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [682]:
col=['age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

In [683]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0].query('TAZ==511')#.to_excel('delet.xlsx')#[col].sum(axis=1)

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check


In [684]:
len(taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0])

0

In [685]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop']>0].loc[taz['hh_total']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check


In [686]:
col=[
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up']

In [687]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['pop']<=0]['TAZ'])
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['hh_total']<=0]['TAZ'])

In [688]:
taz_to_check

[]

In [689]:
col=['indus',
'com_hotel',
'business',
'public',
'education',
'agri']

In [690]:
taz['emp_check']=round(taz[col].sum(axis=1)-taz['emp_tot'])

In [691]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['emp_check']!=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check,emp_check


In [692]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['emp_tot']<=0]['TAZ'])

In [693]:
taz_to_check

[]

In [694]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_emp_employed']>0].loc[taz['pop']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check,emp_check
